# 实验 1.1：环境搭建与模型调用

## 实验目标
- 熟悉 Cloud Studio 实验环境
- 学会加载和调用大语言模型
- 理解模型参数（Temperature）对输出的影响
- 为后续安全实验打下基础

## 实验环境
- 平台：腾讯 Cloud Studio（https://cloudstudio.net/）
- GPU：NVIDIA Tesla T4（16GB 显存）
- 模型：Qwen2-1.5B-Instruct（阿里通义千问）

## 预计时间：20分钟

---

## 第一部分：环境准备与验证

> 💡 **为什么要检查环境？**
> 
> 在进行 AI 实验前，需要确认 GPU 环境是否正确配置。GPU（图形处理器）能让模型推理速度提升 10-100 倍！
> Cloud Studio 免费提供 NVIDIA T4 GPU（16GB 显存），足够运行本课程所有实验。

### 1.1 安装依赖

首先安装本实验需要的 Python 库：

In [ ]:
# 安装必要的依赖（首次运行需要几分钟）
!pip install torch transformers accelerate -q

print("✓ 依赖安装完成！")

### 1.2 验证 GPU 环境

运行下面的代码，检查你的实验环境是否配置正确：

In [ ]:
# ====== 环境验证脚本 ======
import sys
import torch
import transformers

print("=" * 50)
print("🔍 AI 安全实验环境检查")
print("=" * 50)

# 检查 Python 版本
print(f"\n[1] Python 版本: {sys.version.split()[0]}")

# 检查 PyTorch 和 CUDA
print(f"[2] PyTorch 版本: {torch.__version__}")
print(f"    CUDA 可用: {'✓ 是' if torch.cuda.is_available() else '✗ 否'}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"    GPU 型号: {gpu_name}")
    print(f"    GPU 显存: {gpu_memory:.1f} GB")

# 检查 Transformers
print(f"[3] Transformers 版本: {transformers.__version__}")

print("\n" + "=" * 50)
if torch.cuda.is_available():
    print("✅ 环境检查通过！GPU 已就绪，可以开始实验")
else:
    print("⚠️ 警告：未检测到 GPU")
    print("   请在 Cloud Studio 中选择 GPU 环境")
print("=" * 50)

### ✅ 检查点 1

运行上面的代码后，你应该看到：
- [ ] CUDA 可用显示 "✓ 是"
- [ ] GPU 型号显示 "Tesla T4" 或类似
- [ ] GPU 显存约 16 GB

如果显示 "未检测到 GPU"，请确认已在 Cloud Studio 中选择 GPU 环境。

---

## 第二部分：加载中文大语言模型

> 💡 **什么是大语言模型（LLM）？**
> 
> 大语言模型是一种 AI 模型，通过学习海量文本数据，能够理解和生成人类语言。
> 常见的 LLM 包括：ChatGPT、文心一言、通义千问等。

### 为什么选择 Qwen2-1.5B？

| 优势 | 说明 |
|------|------|
| 🇨🇳 中文优秀 | 专门针对中文优化，理解能力强 |
| 💾 资源友好 | 1.5B 参数，T4 GPU 轻松运行（约 4GB 显存） |
| 🔒 安全护栏 | 内置安全机制，适合安全实验 |
| 💬 对话优化 | Instruct 版本经过指令微调，对话体验好 |

### 关键概念

- **Tokenizer（分词器）**：将文本转换为模型能理解的数字序列
- **半精度（float16）**：使用 16 位浮点数存储参数，显存减半，速度更快

In [ ]:
# ====== 加载 Qwen2-1.5B 模型 ======
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 指定模型名称
model_name = "Qwen/Qwen2-1.5B-Instruct"

print(f"正在加载模型: {model_name}")
print("首次加载需要下载模型文件（约 3GB），请耐心等待...")
print()

# ========== 填空 1：加载分词器 ==========
# 
# 🎯 任务：使用 AutoTokenizer 加载分词器
# 
# 💡 提示：
#   - AutoTokenizer 可以自动识别模型类型
#   - 使用 from_pretrained() 方法，传入模型名称
#
# 请将 ___________ 替换为正确的代码

tokenizer = ___________

# 加载模型到 GPU（使用半精度节省显存）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 半精度，显存减半
    device_map="auto"           # 自动选择设备（优先 GPU）
)

print("\n" + "=" * 40)
print("✅ 模型加载成功！")
print(f"   模型参数量: {model.num_parameters() / 1e9:.2f}B（十亿）")
print(f"   运行设备: {next(model.parameters()).device}")
print("=" * 40)

### ✅ 检查点 2

运行上面的代码后，你应该看到：
- [ ] 模型加载成功
- [ ] 参数量约 1.54B
- [ ] 运行设备显示 cuda:0（表示在 GPU 上）

---

## 第三部分：与模型对话

> 💡 **对话格式说明**
> 
> 与 LLM 对话需要按照特定格式构建输入。Qwen 使用 `messages` 格式：
> ```python
> messages = [
>     {"role": "system", "content": "系统提示，定义 AI 角色"},
>     {"role": "user", "content": "用户的问题"}
> ]
> ```
> - **system**：系统提示，设定 AI 的行为规范（用户通常看不到）
> - **user**：用户输入的问题或指令

In [ ]:
# ====== 定义对话函数 ======

def chat(user_message, system_prompt="你是一个有帮助的AI助手。", temperature=0.7):
    """
    与模型进行对话
    
    参数:
        user_message: 用户输入的消息
        system_prompt: 系统提示词（定义AI的角色和行为）
        temperature: 控制输出随机性（0-2，越高越随机）
    
    返回:
        模型的回复文本
    """
    # 构建对话格式
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    # ========== 填空 2：将对话转换为模型输入格式 ==========
    # 
    # 🎯 任务：使用 tokenizer 的 apply_chat_template 方法转换格式
    # 
    # 💡 提示：
    #   - apply_chat_template 将 messages 列表转换为模型需要的文本格式
    #   - 需要设置 tokenize=False（返回文本而非数字）
    #   - 需要设置 add_generation_prompt=True（添加生成提示）
    #
    # 请将 ___________ 替换为正确的代码
    
    text = ___________
    
    # 编码输入并移到 GPU
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # 生成回复
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,      # 最多生成 256 个 token
        temperature=temperature, # 控制随机性
        do_sample=True,          # 启用采样
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 解码输出（只取新生成的部分）
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:], 
        skip_special_tokens=True
    )
    return response

print("✓ 对话函数定义完成！")

In [ ]:
# 测试对话功能
print("🧪 测试对话功能...")
print("=" * 50)

response = chat("你好，请用一句话介绍你自己。")

print(f"👤 用户: 你好，请用一句话介绍你自己。")
print(f"🤖 AI: {response}")
print("=" * 50)

---

## 第四部分：探索模型能力

> 💡 **为什么要了解模型能力？**
> 
> 了解模型的能力边界，是进行安全测试的基础。
> 我们需要知道模型"能做什么"，才能探索它"可能被滥用做什么"。

In [ ]:
# ====== 测试模型的不同能力 ======

test_questions = [
    "什么是人工智能？请用简单的话解释。",
    "用 Python 写一个计算 1 到 100 求和的代码。",
    "帮我写一首关于春天的五言绝句。",
]

print("=" * 60)
print("🧪 模型能力测试")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n【问题 {i}】{question}")
    print("-" * 50)
    
    # ========== 填空 3：调用对话函数 ==========
    # 
    # 🎯 任务：调用 chat() 函数获取模型回复
    # 
    # 💡 提示：
    #   - 使用前面定义的 chat() 函数
    #   - 传入 question 作为参数
    #
    # 请将 ___________ 替换为正确的代码
    
    response = ___________
    
    print(f"【回复】\n{response}")
    print("=" * 60)

### 🤔 思考一下

观察上面的输出，思考以下问题：

1. **能力评估**：模型在哪些任务上表现较好？哪些较弱？
2. **安全视角**：如果有人想让模型做"不该做的事"，会从哪些能力入手？

---

## 第五部分：Temperature 参数实验

> 💡 **什么是 Temperature？**
> 
> Temperature（温度）是控制模型输出随机性的关键参数。
> 你可以把它想象成"创意开关"：

| 温度值 | 特点 | 类比 | 适用场景 |
|--------|------|------|----------|
| 低温（0.1-0.3） | 输出确定、保守、一致 | 严谨的学者 | 代码生成、客服 |
| 中温（0.5-0.7） | 平衡创意与准确性 | 理性的作家 | 通用对话 |
| 高温（0.9-1.5） | 输出随机、有创意、多样 | 天马行空的艺术家 | 创意写作 |

### ⚠️ 安全警示

**高温设置可能导致模型产生意外输出**——这在安全测试中很重要：
- 攻击者可能利用高温设置诱导模型生成不当内容
- 高温下模型更容易"突破"安全护栏

In [ ]:
# ====== Temperature 对比实验 ======

prompt = "给我讲一个关于机器人的故事的开头。"

print(f"📝 测试问题: {prompt}")
print()

# 低温测试
print("=" * 55)
print("❄️ 低温生成 (Temperature = 0.1) - 结果更一致")
print("=" * 55)
for i in range(3):
    result = chat(prompt, temperature=0.1)
    print(f"第 {i+1} 次: {result[:80]}...")

# 高温测试
print()
print("=" * 55)
print("🔥 高温生成 (Temperature = 1.2) - 结果更多样")
print("=" * 55)
for i in range(3):
    result = chat(prompt, temperature=1.2)
    print(f"第 {i+1} 次: {result[:80]}...")

### 🤔 思考一下

1. **观察差异**：低温和高温的输出有什么明显区别？
2. **应用场景**：如果你在开发一个银行客服 AI，应该用高温还是低温？为什么？
3. **安全风险**：高温设置可能带来什么安全风险？

---

## 第六部分：系统提示的安全作用

> 💡 **系统提示（System Prompt）是什么？**
> 
> 系统提示是开发者写给 AI 的"隐藏指令"，定义了 AI 的角色、能力边界和行为规范。
> 
> **关键安全概念**：
> - 系统提示是 AI 安全的"第一道防线"
> - 但它并不是万能的——很多攻击就是试图绕过系统提示的限制
> - 这就是我们后续要学习的"提示词注入攻击"的基础

In [ ]:
# ====== 系统提示对比实验 ======

# ========== 填空 4：定义一个严格限制的系统提示 ==========
# 
# 🎯 任务：创建一个只允许回答编程问题的系统提示
# 
# 💡 提示：
#   - 明确说明 AI 的角色（编程助手）
#   - 明确说明限制（只回答编程问题）
#   - 明确说明对其他问题的处理方式（礼貌拒绝）
#
# 示例格式："你是一个..., 你只能..., 对于其他问题..."

strict_system_prompt = ___________

# 定义其他系统提示用于对比
system_prompts = {
    "默认助手": "你是一个有帮助的AI助手。",
    "安全专家": "你是一位网络安全专家。你热情地回答安全相关问题，对于其他问题，礼貌地引导回安全话题。",
    "严格模式": strict_system_prompt,
}

# 测试不同系统提示的效果
test_message = "今天天气怎么样？"

print(f"📝 测试问题: {test_message}")
print("=" * 60)

for name, prompt in system_prompts.items():
    print(f"\n【{name}】")
    print(f"   系统提示: {prompt[:50]}...")
    print("-" * 50)
    response = chat(test_message, system_prompt=prompt)
    print(f"   回复: {response[:150]}")

### 🤔 思考一下

1. **观察差异**：不同系统提示下，模型对同一问题的回答有什么不同？
2. **有效性**："严格模式"下模型是否成功拒绝了非编程问题？
3. **安全思考**：如果有人想让模型"忽略"系统提示，可能会怎么做？（这就是提示词注入攻击的核心思想！）

---

## 📋 实验小结

### 核心收获

通过本实验，你已经：

| 目标 | 完成情况 |
|------|----------|
| 验证 GPU 环境 | ✅ 确认 T4 GPU 可用 |
| 加载中文 LLM | ✅ 掌握 Tokenizer + Model 加载方式 |
| 理解 Temperature | ✅ 低温一致，高温多样 |
| 理解系统提示 | ✅ 第一道防线，但可能被绕过 |

### 🔑 关键概念回顾

```python
# 1. 加载模型的标准流程
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. 对话格式
messages = [
    {"role": "system", "content": "系统提示"},
    {"role": "user", "content": "用户消息"}
]

# 3. Temperature 的安全影响
# 低温 = 稳定输出，高温 = 可能意外输出
```

### 🔮 预告：安全隐患

本实验我们学习了 LLM 的基本使用，但你可能已经注意到一些潜在问题：

- 系统提示可能被"绕过"？
- 模型可能被诱导生成不当内容？
- 高温设置可能导致意外输出？

这些都是 **AI 安全** 需要关注的问题，我们将在后续模块深入探讨！

---

## ⏭️ 下一步

继续完成 **实验 1.2：AI 漏洞侦察初体验**，学习如何探测模型的安全边界。

---

## 📖 参考答案

<details>
<summary>点击展开参考答案</summary>

**填空 1**：
```python
tokenizer = AutoTokenizer.from_pretrained(model_name)
```

**填空 2**：
```python
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
```

**填空 3**：
```python
response = chat(question)
```

**填空 4**（示例）：
```python
strict_system_prompt = "你是一个专业的编程助手。你只能回答编程和技术相关的问题。对于其他问题，请礼貌地说明你只能帮助解决编程问题。"
```

</details>

In [ ]:
# 实验结束，清理显存（可选）
# 如果要继续进行下一个实验，可以运行这段代码释放显存

# del model, tokenizer
# import torch
# torch.cuda.empty_cache()
# print("✓ 显存已清理")

## 第一部分：环境验证

首先确认 GPU 环境是否正确配置。Cloud Studio 提供免费的 T4 GPU。

In [ ]:
# ====== 环境验证脚本 ======
print("=" * 50)
print("AI 安全实验环境检查")
print("=" * 50)

# 检查 Python 版本
import sys
print(f"\n[1] Python 版本: {sys.version.split()[0]}")

# 检查 PyTorch 和 CUDA
import torch
print(f"[2] PyTorch 版本: {torch.__version__}")
print(f"    CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"    GPU 型号: {torch.cuda.get_device_name(0)}")
    print(f"    GPU 显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 检查 Transformers
import transformers
print(f"[3] Transformers 版本: {transformers.__version__}")

# 检查其他依赖
import numpy as np
import matplotlib
print(f"[4] NumPy 版本: {np.__version__}")
print(f"[5] Matplotlib 版本: {matplotlib.__version__}")

print("\n" + "=" * 50)
if torch.cuda.is_available():
    print("✓ 环境检查通过！GPU 已就绪")
else:
    print("⚠ 警告：未检测到 GPU，请确认已选择 GPU 环境")
print("=" * 50)

---

## 第二部分：加载中文大语言模型

我们使用阿里的 **Qwen2-1.5B-Instruct** 模型，这是目前最优秀的中文开源模型之一。

**为什么选择 Qwen2？**
- 中文能力顶级，专门针对中文优化
- 1.5B 参数，T4 GPU 轻松运行（约 4GB 显存）
- 有安全护栏，适合后续安全实验
- Instruct 版本经过指令微调，对话体验好

In [ ]:
# ====== 加载 Qwen2-1.5B 模型 ======
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2-1.5B-Instruct"

print(f"正在加载模型: {model_name}")
print("首次加载需要下载模型文件（约 3GB），请耐心等待...")

# 【填空 1】加载分词器
# 提示：使用 AutoTokenizer.from_pretrained() 方法，参数是 model_name
# 参考答案：tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = ___________________

# 加载模型到 GPU（使用半精度节省显存）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 半精度，显存减半
    device_map="auto"           # 自动选择设备（GPU）
)

print("\n✓ 模型加载成功！")
print(f"  模型参数量: {model.num_parameters() / 1e9:.2f}B")

---

## 第三部分：与模型对话

现在让我们定义一个对话函数，与模型进行交互。

In [ ]:
# ====== 定义对话函数 ======

def chat(user_message, system_prompt="你是一个有帮助的AI助手。"):
    """
    与模型进行对话
    
    参数:
        user_message: 用户输入的消息
        system_prompt: 系统提示词（定义AI的角色和行为）
    
    返回:
        模型的回复文本
    """
    # 构建对话格式（Qwen 使用 messages 格式）
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    # 【填空 2】将对话转换为模型输入格式
    # 提示：使用 tokenizer.apply_chat_template() 方法
    # 参考答案：text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    text = ___________________
    
    # 编码输入并移到 GPU
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # 生成回复
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 解码输出（只取新生成的部分）
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:], 
        skip_special_tokens=True
    )
    return response

# 测试对话
print("测试对话功能...")
response = chat("你好，请用一句话介绍你自己。")
print(f"\n用户: 你好，请用一句话介绍你自己。")
print(f"AI: {response}")

---

## 第四部分：探索模型能力

In [ ]:
# ====== 测试模型的不同能力 ======

test_questions = [
    "什么是人工智能？请用简单的话解释。",
    "用 Python 写一个计算 1 到 100 求和的代码。",
    "帮我写一首关于春天的五言绝句。",
]

print("=" * 60)
print("模型能力测试")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n【问题 {i}】{question}")
    print("-" * 40)
    
    # 【填空 3】调用对话函数获取回复
    # 提示：调用前面定义的 chat() 函数，参数是 question
    # 参考答案：response = chat(question)
    response = ___________________
    
    print(f"【回复】\n{response}")
    print("=" * 60)

---

## 第五部分：Temperature 参数实验

Temperature 控制输出的随机性：
- **低温（如 0.1）**：输出更确定、更保守、更一致
- **高温（如 1.2）**：输出更随机、更有创意、更多样

这个参数在安全测试中很重要：高温设置可能导致模型产生意外输出。

In [ ]:
# ====== Temperature 对比实验 ======

def chat_with_temp(message, temperature):
    """使用指定 temperature 进行对话"""
    messages = [
        {"role": "system", "content": "你是一个有帮助的AI助手。"},
        {"role": "user", "content": message}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

prompt = "给我讲一个关于机器人的故事的开头。"

print(f"测试问题: {prompt}")
print("\n" + "=" * 50)
print("低温生成 (Temperature = 0.1) - 结果更一致")
print("=" * 50)
for i in range(3):
    result = chat_with_temp(prompt, temperature=0.1)
    print(f"第 {i+1} 次: {result[:80]}...")

print("\n" + "=" * 50)
print("高温生成 (Temperature = 1.2) - 结果更多样")
print("=" * 50)
for i in range(3):
    result = chat_with_temp(prompt, temperature=1.2)
    print(f"第 {i+1} 次: {result[:80]}...")

### 观察问题

1. 低温和高温的输出有什么区别？
2. 如果你在开发一个银行客服 AI，应该用高温还是低温？为什么？
3. 高温设置可能带来什么安全风险？

---

## 第六部分：观察系统提示的作用

**系统提示（System Prompt）** 定义了 AI 的角色和行为规范。

这在安全测试中非常重要——很多攻击就是试图绕过系统提示的限制。

In [ ]:
# ====== 系统提示对比实验 ======

system_prompts = {
    "默认助手": "你是一个有帮助的AI助手。",
    "安全专家": "你是一位网络安全专家，专门回答安全相关的问题。对于非安全问题，礼貌地引导用户回到安全话题。",
    "严格模式": "你是一个严格的AI助手。你只能回答编程和技术问题，对于其他问题你必须拒绝回答。",
}

test_message = "今天天气怎么样？"

print(f"测试问题: {test_message}")
print("=" * 60)

for name, prompt in system_prompts.items():
    print(f"\n【{name}】")
    print(f"系统提示: {prompt[:50]}...")
    print("-" * 40)
    response = chat(test_message, system_prompt=prompt)
    print(f"回复: {response}")

### 观察问题

1. 不同系统提示下，模型对同一问题的回答有什么不同？
2. "严格模式"下模型是否成功拒绝了非技术问题？
3. 如果有人想让模型忽略系统提示，可能会怎么做？（这就是提示词注入攻击的核心）

---

## 实验总结

通过本实验，你应该：

✅ 熟悉了 Cloud Studio 的 GPU 环境

✅ 学会了加载和调用 Qwen2 中文大语言模型

✅ 理解了 Temperature 参数对输出的影响

✅ 观察了系统提示如何控制模型行为

### 关键收获

1. **Temperature 影响安全性**：高温可能导致意外输出
2. **系统提示是第一道防线**：但它可能被攻击者绕过
3. **中文模型选择很重要**：Qwen2 是目前最佳的中文开源选择

---

## 下一步

继续完成 **实验 1.2：AI 漏洞侦察**，学习如何探测模型的安全边界。

---

## 参考答案

**填空 1**：`tokenizer = AutoTokenizer.from_pretrained(model_name)`

**填空 2**：`text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)`

**填空 3**：`response = chat(question)`

In [ ]:
# 清理显存（实验结束后运行）
# del model, tokenizer
# torch.cuda.empty_cache()
# print("显存已清理")